<a href="https://colab.research.google.com/github/BibhuDas123/Statistics-Behind-Data-Science/blob/main/Custom_NER_building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import spacy
nlp=spacy.load('en_core_web_sm')
nlp.pipe_names

['tagger', 'parser', 'ner']

In [ ]:
doc=nlp('Australia wants to force Facebook and Google to pay media companies for news')

In [ ]:
for ent in doc.ents:
  print(ent.text,ent.start_char,ent.end_char,ent.label_)

Australia 0 9 GPE
Facebook and Google 25 44 ORG


In [ ]:
doc=nlp("I do not money to pay my credit card account")

In [ ]:
for ent in doc.ents:
  print(ent.text,ent.start_char,ent.end_char,ent.label_)

In [ ]:
train = [
         ("Money transfer from my checking account is not working", {"entities": [(6, 13, "ACTIVITY"), (23, 39, 'PRODUCT')]}),
         ("I want to check balance in my savings account", {"entities": [(16, 23, "ACTIVITY"), (30, 45, 'PRODUCT')]}),
         ("I suspect a fraud in my credit card account", {"entities": [(12, 17, "ACTIVITY"), (24, 35, 'PRODUCT')]}),
         ("I am here for opening a new savings account", {"entities": [(14, 21, "ACTIVITY"), (28, 43, 'PRODUCT')]}),
         ("Your mortgage is in delinquent status", {"entities": [(20, 30, "ACTIVITY"), (5, 13, 'PRODUCT')]}),
         ("Your credit card is in past due status", {"entities": [(23, 31, "ACTIVITY"), (5, 16, 'PRODUCT')]}),
         ("My loan account is still not approved and funded", {"entities": [(25, 37, "ACTIVITY"), (3, 15, 'PRODUCT'), (42, 48, "ACTIVITY")]}),
         ("How do I open a new loan account", {"entities": [(9, 13, "ACTIVITY"), (20, 32, 'PRODUCT')]}),
         ("What are the charges on Investment account", {"entities": [(13, 20, "ACTIVITY"), (24, 42, 'PRODUCT')]}),
         ("Can you explain late charges on my credit card", {"entities": [(21, 28, "ACTIVITY"), (35, 46, 'PRODUCT')]}),
         ("I want to open a new loan account", {"entities": [(10, 14, "ACTIVITY"), (21, 33, 'PRODUCT')]}),
         ("Can you help updating payment on my credit card", {"entities": [(22, 29, "ACTIVITY"), (36, 47, 'PRODUCT')]}),
         ("When is the payment due date on my card", {"entities": [(12, 19, "ACTIVITY"), (35, 39, 'PRODUCT')]})
        ]

In [ ]:
nlp.pipe_names

['tagger', 'parser', 'ner']

In [ ]:
ner=nlp.get_pipe("ner")

In [ ]:
for _,annotations in train:
  for ent in annotations.get('entities'):
    ner.add_label(ent[2])

(6, 13, 'ACTIVITY')
(23, 39, 'PRODUCT')
(16, 23, 'ACTIVITY')
(30, 45, 'PRODUCT')
(12, 17, 'ACTIVITY')
(24, 35, 'PRODUCT')
(14, 21, 'ACTIVITY')
(28, 43, 'PRODUCT')
(20, 30, 'ACTIVITY')
(5, 13, 'PRODUCT')
(23, 31, 'ACTIVITY')
(5, 16, 'PRODUCT')
(25, 37, 'ACTIVITY')
(3, 15, 'PRODUCT')
(42, 48, 'ACTIVITY')
(9, 13, 'ACTIVITY')
(20, 32, 'PRODUCT')
(13, 20, 'ACTIVITY')
(24, 42, 'PRODUCT')
(21, 28, 'ACTIVITY')
(35, 46, 'PRODUCT')
(10, 14, 'ACTIVITY')
(21, 33, 'PRODUCT')
(22, 29, 'ACTIVITY')
(36, 47, 'PRODUCT')
(12, 19, 'ACTIVITY')
(35, 39, 'PRODUCT')


In [ ]:
disable_pipes=[pipe for pipe in nlp.pipe_names if pipe!="ner"]

In [ ]:
#To train in minibatch and disable pipes to diable the disable other components of pipe(tagger,parser).We are using existing pipeline but if we want we can build from scratch
# If we build in scatrch then build nlp.entity.We are reunning for 100 iterations and shuffle the data and we are creatinf the minibatch with compounding or do record by record
import random
from spacy.util import minibatch,compounding
from pathlib import Path

with nlp.disable_pipes(*disable_pipes):
  optimizer=nlp.resume_training()

  for iteration in range(100):
    random.shuffle(train)
    losses={}

    batches=minibatch(train,size=compounding(1.0,4.0,1.001))
    for batch in batches:
      text,annotation=zip(*batch)
      nlp.update(
          text,
          annotation,
          drop=0.5,
          losses=losses,
          sgd=optimizer
      )
      print("Losses",losses)

Losses {'ner': 6.1761456898800295}
Losses {'ner': 14.072928793149146}
Losses {'ner': 19.87590928845421}
Losses {'ner': 29.1222560804446}
Losses {'ner': 34.46088907629124}
Losses {'ner': 44.08393797784916}
Losses {'ner': 51.83643149004617}
Losses {'ner': 60.15313741253976}
Losses {'ner': 67.76743396945992}
Losses {'ner': 74.32217888758484}
Losses {'ner': 81.94242673691357}
Losses {'ner': 88.05782811206126}
Losses {'ner': 97.79883550993367}
Losses {'ner': 9.592891931533813}
Losses {'ner': 18.064947785868526}
Losses {'ner': 24.4313976733848}
Losses {'ner': 33.895987002294575}
Losses {'ner': 41.78302588816818}
Losses {'ner': 46.82266918746841}
Losses {'ner': 54.79482504791899}
Losses {'ner': 62.34049135035143}
Losses {'ner': 68.68663547071611}
Losses {'ner': 78.937104177477}
Losses {'ner': 86.93591439805952}
Losses {'ner': 99.3899906285855}
Losses {'ner': 110.62832301937071}
Losses {'ner': 12.814996108412743}
Losses {'ner': 19.668491896241903}
Losses {'ner': 31.577910777181387}
Losses {'ne

In [ ]:
for text,_ in train:
  doc=nlp(text)
  print('Entities',[(ent.text,ent.label_) for ent in doc.ents])

Entities [('checking account', 'PRODUCT')]
Entities [('charges', 'ACTIVITY'), ('credit card', 'PRODUCT')]
Entities [('charges', 'ACTIVITY'), ('Investment account', 'PRODUCT')]
Entities [('fraud', 'ACTIVITY'), ('credit card', 'PRODUCT')]
Entities [('open', 'ACTIVITY'), ('loan account', 'PRODUCT')]
Entities [('payment', 'ACTIVITY'), ('credit card', 'PRODUCT')]
Entities [('open', 'ACTIVITY'), ('loan account', 'PRODUCT')]
Entities [('payment', 'ACTIVITY')]
Entities [('loan account', 'PRODUCT'), ('funded', 'ACTIVITY')]
Entities [('opening', 'ACTIVITY'), ('savings account', 'PRODUCT')]
Entities [('balance', 'ACTIVITY'), ('savings account', 'PRODUCT')]
Entities [('credit card', 'PRODUCT'), ('past', 'ACTIVITY')]
Entities [('mortgage', 'PRODUCT'), ('delinquent', 'ACTIVITY')]


In [ ]:
from spacy import displacy
doc=nlp("what is the process to open a new savings account")
for ent in doc.ents:
  print(ent.text,ent.start_char,ent.end_char,ent.label_)
displacy.render(nlp(doc.text),style='ent',jupyter=True)

open 23 27 ACTIVITY
savings account 34 49 PRODUCT


In [ ]:
doc=nlp("My credit card payment will be delayed")
for ent in doc.ents:
  print(ent.text,ent.start_char,ent.end_char,ent.label_)

credit card 3 14 PRODUCT


In [ ]:
doc=nlp('what are the charges on credit card late payment in Bank of America')
for ent in doc.ents:
  print(ent.text,ent.start_char,ent.end_char,ent.label_)
displacy.render(nlp(doc.text),style='ent',jupyter=True)

charges 13 20 ACTIVITY
credit card 24 35 PRODUCT
payment 41 48 ACTIVITY


In [ ]:
doc=nlp('My credit card payment will be delayed')
for ent in doc.ents:
  print(ent.text,ent.start_char,ent.end_char,ent.label_)

credit card 3 14 PRODUCT
